<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción
El siguiente cuaderno realiza la suma de dos vectores en forma secuencial, utilizando el procesador CPU. El algoritmo se basa en la función **axpy** [3] de nivel 1, de la biblioteca BLAS [4] que resuelve la ecuación:

Su objetivo es enseñar a los alumnos como se utiliza Python[2] la plataforma Colab [1] y la programación secuencial.

#2. Armado de ambiente

Instala en el cuaderno el módulo CUDA de Python.

In [4]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 5.5MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621047 sha256=3ce2641f805abb76b009e0ae7023b0751a0add490aad5612a387ad07dae2adb7
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=e9bfd0c343845629ecbfdd189306407224c92cd606325a1231706351a7df2c1e
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo

Ejecuta el Código CPU - GPU.

In [7]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   50#@param {type: "number"}
alfa =   1#@param {type: "number"}
# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# CPU - Defino la memoria de los vectores en cpu.
x_cpu = numpy.random.randn( cantidad_N )
x_cpu = x_cpu.astype( numpy.float32() )

y_cpu = numpy.random.randn( cantidad_N )
y_cpu = y_cpu.astype( numpy.float32() )

#tiempo_ini_cpu = datetime.now()

r_cpu = numpy.empty_like( x_cpu )

# CPU - reservo la memoria GPU.
x_gpu = cuda.mem_alloc( x_cpu.nbytes )
y_gpu = cuda.mem_alloc( y_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( x_gpu, x_cpu )
cuda.memcpy_htod( y_gpu, y_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_axpy( int n, float alfa, float *X, float *Y )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    Y[idx]  = alfa*X[idx] + Y[idx];
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_axpy")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N),numpy.float32(alfa), x_gpu, y_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( r_cpu, y_gpu )

"""
# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "Y: " )
print( y_cpu )
print( "------------------------------------")
print( "R: " )
print( r_cpu )
"""

tiempo_total = datetime.now() - tiempo_total

print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

RuntimeError: ignored

#4. Tabla de pasos

Procesador | Funcion | Detalle
------------ | ------------- | -------------
CPU | @param | Lectura del tamaño de vectories de Colab.
CPU | import | Importa los módulos para funcionar.
CPU | datetime.now() | Toma el tiempo actual.
CPU | numpy.random.randn( Cantidad_N ) | Inicializa los vectores A, B y R.
**GPU** | cuda.mem_alloc() | Reserva la memoria en GPU.
**GPU** | cuda.memcpy_htod() | Copia las memorias desde el CPU al GPU.
CPU | SourceModule() | Define el código del Kernel
CPU | module.get_function() | Genera la función del Kernel GPU.
CPU | dim_tx/dim_bx | Calcula las dimensiones.
**GPU** |kernel() | Ejecuta el kernel en GPU
CPU | cuda.memcpy_dtoh() | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU | print() | Informo los resultados.

#5. Conclusiones

Las conclusiones son explicadas en clase....

#6. Bibliografía

[1] MARKDOWN SYNTAX Colab: PDF

[2] Introducción a Python: Página Colab

[3] Función Axpy de biblioteca BLAS: Referencia

[4] Biblioteca BLAS: Referencia

[5] Documentación PyCUDA: WEB

[6] Repositorio de PyCUDA: WEB